# Introduction

This notebook visualizes the results of the spatial auto-correlation analysis carried out in this [notebook: 3b_spatial_auto_correlation](3b_spatial_auto_correlation.ipynb).

NOTE: The only purpose of this notebook is to make it easy to visually inspect the outputs from the previous notebook, it does not write any files to disk

# Imports

In [ ]:
import numpy as np
from skimage import morphology as sk_morph
import cv2
from skimage.morphology import erosion, disk, closing
import os
import json

from matplotlib import pyplot as plt
font = {'size'   : 16}
plt.rc('font', **font)
import matplotlib.patches as patches

from fam13a import utils, register

# Constants

In [ ]:
PROJ_ROOT = utils.here()
# declare the data input directory
HBEC_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'hbec')
# print list of experiment IDs
print(os.listdir(HBEC_ROOT))

In [ ]:
# choose experiment data to load
EXP_ID = 'ELN19575-4'

# declare the various output directories
PROCESSED_ROOT = os.path.join(HBEC_ROOT, EXP_ID)
MAX_FRAME_ROOT = os.path.join(PROCESSED_ROOT, 'max_frame')
REGISTER_ROOT = os.path.join(PROCESSED_ROOT, 'register')
SAC_ROOT = os.path.join(PROCESSED_ROOT, 'spatial_auto_correlation', 'neighbor_size_9_geary')

# find all relevant data files in the data directory 
file_ids = sorted([_d for _d in os.listdir(SAC_ROOT) if os.path.isdir(os.path.join(SAC_ROOT, _d))])
# print available data file IDs
print(file_ids)

# Load Data

In [ ]:
# choose data file to load
_file_id = 'NT_4_13_MMStack_Pos0'
REGISTER_FILE_ROOT = os.path.join(REGISTER_ROOT, _file_id)
SAC_FILE_ROOT = os.path.join(SAC_ROOT, _file_id)

# load each of the associated registration interim/processed files
shifts = np.load(os.path.join(REGISTER_FILE_ROOT, 'shifts.npy'))
max_frame = np.load(os.path.join(MAX_FRAME_ROOT, f'{_file_id}.npy'))
sub_max_frame = np.load(os.path.join(REGISTER_FILE_ROOT, 'max_frame.npy'))
sub_movement_mask = np.load(os.path.join(REGISTER_FILE_ROOT, 'mask.npy'))
roi = np.load(os.path.join(REGISTER_FILE_ROOT, 'roi.npy'))
with open(os.path.join(REGISTER_FILE_ROOT, 'params.json'), 'r') as params_file:
    register_params = json.load(params_file)
    
# load each of the associated spatial autocorrelation processed files
# get matrices from geary objects
gearys_p_speed = np.load(os.path.join(SAC_FILE_ROOT, 'p_values_speed.npy'))
gearys_i_speed = np.load(os.path.join(SAC_FILE_ROOT, 'i_values_speed.npy'))

gearys_p_vx = np.load(os.path.join(SAC_FILE_ROOT, 'p_values_vx.npy'))
gearys_i_vx = np.load(os.path.join(SAC_FILE_ROOT, 'i_values_vx.npy'))
gearys_p_vy = np.load(os.path.join(SAC_FILE_ROOT, 'p_values_vy.npy'))
gearys_i_vy = np.load(os.path.join(SAC_FILE_ROOT, 'i_values_vy.npy'))

# Process

we explicitly set the min and max values, of the colour map, for the vector magnitudes. This is done so the vector field plots of different videos can be compared visually, otherwise matplotlib automatically uses the min and max values present in the image to set the range of the colour map

In [ ]:
min_mag, max_mag = register.calculate_min_max_speed(register_params['ptrn_size'], register_params['region_size'])

In [ ]:
# calculate average velocity field from shifts array
velocity_fields = register.calculate_mean_velocity_field(shifts)
norm_shifts = velocity_fields['normalised_velocity']
mags = velocity_fields['speed']

# for the p-values, we are only interested in those regions where p<threshold
# so here we can convert the p-value matrices to boolean p-value masks
p_value_threshold = 0.05
gearys_p_speed_mask = gearys_p_speed < p_value_threshold;
gearys_p_vx_mask = gearys_p_vx < p_value_threshold;
gearys_p_vy_mask = gearys_p_vy < p_value_threshold;

# for the i-values, we can look at areas of spatial clustering
# i value < 2 indicates that a feature has neighboring features with similarly high or low attribute values; this feature is part of a cluster.
# i value > 2 indicates that a feature has neighboring features with dissimilar values; this feature is an outlier.
# In either instance, the p-value for the feature must be small enough for the cluster or outlier to be considered statistically significant. 
gearys_i_speed_mask = gearys_i_speed < 2
gearys_i_vx_mask = gearys_i_vx < 2
gearys_i_vy_mask = gearys_i_vy < 2

# now we can combine a low p-value with positive i-value to get a mask for statistically significant correlated motion
geary_speed_mask = gearys_i_speed_mask & gearys_p_speed_mask
geary_vx_mask = gearys_i_vx_mask & gearys_p_vx_mask
geary_vy_mask = gearys_i_vy_mask & gearys_p_vy_mask

min_mags_mask = (mags > np.average(mags[~sub_movement_mask & roi]))

# apply roi to mags, norm_shifts, p values and i values
for i in [
    mags, norm_shifts[:,:,0], norm_shifts[:,:,1], 
    gearys_p_speed, gearys_i_speed, gearys_p_vx, gearys_i_vx,
    gearys_p_vy, gearys_i_vy, gearys_i_vy_mask, gearys_i_vx_mask, 
    gearys_i_speed_mask, geary_speed_mask, geary_vx_mask, geary_vy_mask,
    gearys_p_speed_mask,gearys_p_vx_mask,gearys_p_vy_mask
]:
    i[~roi] = None
coordination_mask = (geary_vx_mask & geary_vy_mask) & sub_movement_mask & min_mags_mask
dilated_coordination_mask = closing(coordination_mask.astype(np.int8), disk(1)).astype(bool)
closing_coordination_mask = sk_morph.closing(coordination_mask.astype(np.int8), disk(1)).astype(bool)

# apply erosion to sub_movement_mask
erosion_mask = cv2.erode(sub_movement_mask.astype(np.uint8), disk(3), iterations = 2).astype(bool)


In [ ]:
plt.figure(figsize=(20, 15))

plt.subplot(341); utils.imshow(sub_movement_mask); plt.axis('off'); plt.title("motion ")
plt.subplot(342); utils.imshow(mags); plt.axis('off'); plt.title("|v|")
plt.subplot(343); utils.imshow(norm_shifts[...,1]); plt.axis('off'); plt.title("vx")
plt.subplot(344); utils.imshow(norm_shifts[...,0]); plt.axis('off'); plt.title("vy")

plt.subplot(345); utils.imshow(coordination_mask); plt.axis('off'); plt.title("basic coordination")
plt.subplot(346); utils.imshow(geary_speed_mask); plt.axis('off'); plt.title("geary |v|")
plt.subplot(347); utils.imshow(geary_vx_mask); plt.axis('off'); plt.title("geary vx")
plt.subplot(348); utils.imshow(geary_vy_mask); plt.axis('off'); plt.title("geary vy")

plt.subplot(3,4,9); utils.imshow(closing_coordination_mask); plt.axis('off'); plt.title("closing coordination")
plt.subplot(3,4,10); utils.imshow(roi); plt.axis('off'); plt.title("region of interest")
plt.subplot(3,4,12); utils.imshow(erosion_mask); plt.axis('off'); plt.title("eroded coordination")

# Visualise

In [ ]:
slices_1 = (slice(225,275), slice(100,150))

# plot full view
plt.figure(figsize=(30, 30))

plt.subplot(121)
plt.title('Speed')
plt.imshow(sub_movement_mask)
plt.quiver(norm_shifts[...,1], norm_shifts[...,0], mags, scale=1, scale_units='xy', angles='xy',
          headwidth=4, headlength=4, headaxislength=3, cmap='hot')
plt.clim(min_mag, max_mag)
ax = plt.gca()
rect_1 = patches.Rectangle((slices_1[1].start,slices_1[0].start),
                 slices_1[1].stop - slices_1[1].start,
                 slices_1[0].stop - slices_1[0].start,
                 linewidth=4,
                 edgecolor='cyan',
                 fill = False)
ax.add_patch(rect_1)

plt.subplot(122)
plt.title('dilated_coordination_mask')
plt.imshow(dilated_coordination_mask)
plt.colorbar(fraction=0.01)
plt.quiver(norm_shifts[...,1], norm_shifts[...,0], mags, scale=1, scale_units='xy', angles='xy',
          headwidth=4, headlength=4, headaxislength=3, cmap='hot')
plt.clim(min_mag, max_mag)
ax = plt.gca()
rect_1 = patches.Rectangle((slices_1[1].start,slices_1[0].start),
                 slices_1[1].stop - slices_1[1].start,
                 slices_1[0].stop - slices_1[0].start,
                 linewidth=4,
                 edgecolor='cyan',
                 fill = False)
ax.add_patch(rect_1)

plt.show()

In [ ]:
slices=slices_1

In [ ]:
# plot a zoomed in view speed vs mi speed
plt.figure(figsize=(30, 30))

plt.subplot(121)
plt.title('movement_mask')
plt.imshow(sub_movement_mask[slices])
plt.quiver(norm_shifts[slices][...,1], norm_shifts[slices][...,0], mags[slices], scale=1, scale_units='xy', angles='xy',
          headwidth=4, headlength=4, headaxislength=3, cmap='hot')
plt.clim(min_mag, max_mag)

plt.subplot(122)
plt.title('coordination_mask')
plt.imshow(closing_coordination_mask[slices])
plt.quiver(norm_shifts[slices][...,1], norm_shifts[slices][...,0], mags[slices], scale=1, scale_units='xy', angles='xy',
          headwidth=4, headlength=4, headaxislength=3, cmap='hot')
plt.clim(min_mag, max_mag)